#### Object Detection Using Multi-Label Classification 

In [ ]:
#Importing Libraries
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape

In [ ]:
# Example paths to images and annotations directories
images_path = 'D:/FoodSeg103/Dataset/img_dir'
annotations_path = 'D:/FoodSeg103/Dataset/ann_dir'
img_size = (128, 128)

# Function to load images and annotations
def load_images_and_annotations(images_path, annotations_path):
    images = []
    labels = []
    
    # List all image files in images_path
    image_files = sorted(os.listdir(images_path))
    
    # Process each image file and its corresponding annotation
    for image_name in image_files:
        image_path = os.path.join(images_path, image_name)
        annotation_path = os.path.join(annotations_path, image_name.replace('.jpg', '.png'))
        
        # Read the input image
        image = cv2.imread(image_path)
        image = cv2.resize(image, img_size)  # Resize image to fixed size
        images.append(image)
        
        # Read the annotation image
        annotation = cv2.imread(annotation_path, cv2.IMREAD_GRAYSCALE)
        annotation = cv2.resize(annotation, img_size, interpolation=cv2.INTER_NEAREST)  # Resize annotation image to fixed size
        unique_labels = np.unique(annotation)  # Get unique labels from annotation
        
        # Convert to label format (assuming binary mask here)
        label = np.zeros_like(annotation, dtype=np.uint8)
        for unique_label in unique_labels:
            label[annotation == unique_label] = 1
        
        labels.append(label)
    
    return images, labels

In [ ]:
# Load images and annotations
images, labels = load_images_and_annotations(images_path, annotations_path)

# Convert lists to numpy arrays
X = np.array(images, dtype='float32') / 255.0  # Normalize pixel values
y = np.array(labels, dtype='float32')  # Use float32 for labels

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Print dataset shapes for verification
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# Define input shape and number of classes based on dataset
input_shape = X_train[0].shape  
num_classes = y_train.shape[-1]  

In [ ]:
# Create an instance of the CNN model
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dropout(0.3), 
    Dense(128 * 128, activation='sigmoid'), 
    Reshape((128, 128))  
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Binary cross-entropy for multi-label classification
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test) 
print(f"Test Accuracy: {accuracy}")

In [ ]:
# Make predictions on the test set
predictions = model.predict(X_test)

# Threshold the predictions to get binary outputs
threshold = 0.5
binary_predictions = (predictions > threshold).astype(int)

# Reshape the binary predictions to match the shape of the ground truth
binary_predictions = binary_predictions.reshape(-1, 128, 128)
y_test = y_test.reshape(-1, 128, 128)